In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood

import numpy as np
import pandas as pd
from gumps.studies.batch_study import AbstractBatchStudy

from gumps.solvers.sampler import SamplerSolverParameters
from gumps.apps.parametric_sweep import ParametricSweepApp

import sklearn.preprocessing

/home/wheymann/mambaforge/envs/gumps_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create a batch study

In [2]:
model_variables = {}

class MultiBatch(AbstractBatchStudy):
    "batch version of sphere study (designed to approximate surrogate model)"

    def __init__(self, model_variables:dict):
        self.model_variables = model_variables

    def start(self):
        "initialize this study"

    def stop(self):
        "handle shutdown tasks"

    def run(self, input_data:pd.DataFrame, processing_function) -> pd.DataFrame:
        "run the batch simulation"
        diff = pd.DataFrame({'d1': input_data['a']**2 + input_data['b']**2,
                            'd2': input_data['b']**2 + input_data['c']**2,
                            'd3': input_data['c']**2 + input_data['a']**2,
                            'd4': input_data['a']**2 + input_data['b']**2 + input_data['c']**2})
        self.save_results(input_data, diff)

        return processing_function(diff)

batch = MultiBatch(model_variables=model_variables)

# Sample the batch study

In [3]:
parameters = SamplerSolverParameters(
    number_of_samples = 128,
    lower_bound = {'a':-10, 'b':-10, 'c':-10},
    upper_bound = {'a':10,  'b':10, 'c':10},
    sampler = "sobol"
    )

def get_all(frame:pd.DataFrame) -> pd.DataFrame:
    "processing function to get the total from the dataframe"
    return frame

app = ParametricSweepApp(parameters=parameters,
        processing_function=get_all,
        pre_processing_function=None,
        directory=None,
        batch=batch)
app.run()

# Setup the optimizer

In [4]:
input_scaler = sklearn.preprocessing.MinMaxScaler()
output_scaler = sklearn.preprocessing.StandardScaler()

train_X = input_scaler.fit_transform(app.factors.to_numpy())
train_Y = output_scaler.fit_transform(app.responses.to_numpy())

train_X = torch.DoubleTensor(train_X)
train_Y = torch.DoubleTensor(train_Y)


models = []
for i in range(train_Y.shape[-1]):
    models.append(
        SingleTaskGP(
            train_X, train_Y[..., i : i + 1]
        )
    )
gp = ModelListGP(*models)
mll = SumMarginalLogLikelihood(gp.likelihood, gp)

fit_gpytorch_mll(mll)

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0-3): 4 x GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0-3): 4 x SingleTaskGP(
        (likelihood): GaussianLikelihood(
          (noise_covar): HomoskedasticNoise(
            (noise_prior): GammaPrior()
            (raw_noise_constraint): GreaterThan(1.000E-04)
          )
        )
        (mean_module): ConstantMean()
        (covar_module): ScaleKernel(
          (base_kernel): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
          (outputscale_prior): GammaPrior()
          (raw_outputscale_constraint): Positive()
        )
      )
    )
    (likelihood): LikelihoodList(
      (likelihoods): ModuleList(
        

In [54]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1, maximize=False)

UnsupportedError: Must specify a posterior transform when using a multi-output model.

In [37]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(5), torch.ones(5)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=10, raw_samples=50
)
candidate  # tensor([0.4887, 0.5063])

tensor([[0.5069, 0.5090, 0.4701, 0.3914, 0.4506]])

In [38]:
input_scaler.inverse_transform(candidate.detach().numpy().reshape(1, -1))

array([[ 0.18668982,  0.3207016 , 19.466476  ,  0.06941879,  4.5460224 ]],
      dtype=float32)